## Key pair generation

## Schnorr Sign

- M message, d is the private key, P = dG is the public key
- A, the signer, picks a random $k, 1 < k < n$ and calculates $R = kG$
- A calculates $e = h(R || P || M)$
- A calculates $s = (k + ed) \mod n$
- The signature is the pair $(R_x, s)$ 

In [1]:
print("ciao")


ciao


## Verify

- B, the receiver, calculates $R_{v} = sG$
- B calculates $e = h(R || P || M)$
- if $R_{v} - R = eP$ the signature is valid

# Schnorr MuSig-1



# Schnorr MuSig-2



# Verify MuSig-1 and MuSig-2